In [1]:
import pandas
import json
from statistics import mean

In [2]:
df = pandas.read_json('data.json',lines=True)

# Exploratory Data Analysis (EDA)

In [3]:
df.shape

(9020, 13)

In [4]:
df.dtypes

visitNumber               int64
visitId                   int64
visitStartTime            int64
date                      int64
totals                   object
trafficSource            object
device                   object
geoNetwork               object
customDimensions         object
hits                     object
fullVisitorId           float64
channelGrouping          object
socialEngagementType     object
dtype: object

In [5]:
df.head()

,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,channelGrouping,socialEngagementType
0,4,1494216917,1494216917,20170507,"{'visits': '1', 'hits': '37', 'pageviews': '24...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Oceania', 'subContinent': 'Aust...","[{'index': '4', 'value': 'APAC'}]","[{'hitNumber': '1', 'time': '0', 'hour': '21',...",5.162435e+18,Referral,Not Socially Engaged
1,1,1498312914,1498312914,20170624,"{'visits': '1', 'hits': '25', 'pageviews': '17...","{'referralPath': '/yt/about/press/', 'campaign...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': '4', 'value': 'EMEA'}]","[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",1.732367e+18,Social,Not Socially Engaged
2,1,1498347122,1498347122,20170624,"{'visits': '1', 'hits': '17', 'pageviews': '12...","{'campaign': '(not set)', 'source': '(direct)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': '4', 'value': 'North America'}]","[{'hitNumber': '1', 'time': '0', 'hour': '16',...",1.432953e+18,Organic Search,Not Socially Engaged
3,1,1488025415,1488025415,20170225,"{'visits': '1', 'hits': '36', 'pageviews': '22...","{'campaign': '(not set)', 'source': 'google', ...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': '4', 'value': 'EMEA'}]","[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",9.675831e+18,Organic Search,Not Socially Engaged
4,1,1488051271,1488051271,20170225,"{'visits': '1', 'hits': '83', 'pageviews': '65...","{'campaign': '(not set)', 'source': 'google', ...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': '4', 'value': 'North America'}]","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",3.973743e+18,Organic Search,Not Socially Engaged


# Creating an unique ID

In [6]:
list_ids = [str(c[1]['fullVisitorId']) + '#' + str(c[1]['visitId']) for c in df.iterrows()]
df['NewId'] = list_ids

In [7]:
df.columns

Index(['visitNumber', 'visitId', 'visitStartTime', 'date', 'totals',
       'trafficSource', 'device', 'geoNetwork', 'customDimensions', 'hits',
       'fullVisitorId', 'channelGrouping', 'socialEngagementType', 'NewId'],
      dtype='object')

# Total Page Views

In [8]:
totals = df.totals.to_json(orient='records')
devices = df.device.to_json(orient='records')

totals_json = json.loads(totals)
devices_json = json.loads(devices)

df2 = pandas.concat([df,pandas.io.json.json_normalize(totals_json),pandas.io.json.json_normalize(devices_json)],axis=1)

df2.drop(columns = ['totals','device'],inplace=True)

C:\Users\amori\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [9]:
totalpageviews = df2['pageviews'].fillna(0).astype(int).sum()
print(f'The number of views is {totalpageviews}')

The number of views is 35737


In [10]:
df['fullVisitorId']

0       5.162435e+18
1       1.732367e+18
2       1.432953e+18
3       9.675831e+18
4       3.973743e+18
            ...     
9015    5.368524e+17
9016    6.400230e+17
9017    9.590960e+18
9018    2.136266e+18
9019    5.537481e+18
Name: fullVisitorId, Length: 9020, dtype: float64

# Number of Sessions by fullVisitorId

In [11]:
df2.groupby(['fullVisitorId']).count()['NewId'].to_frame()

,NewId
fullVisitorId,
1.701872e+14,2
1.799242e+14,1
4.923041e+14,1
6.967907e+14,1
8.146259e+14,1
...,...
9.997026e+18,1
9.997962e+18,1
9.998281e+18,1


In [12]:
df2['timeOnSite'] = df2['timeOnSite'].fillna(0).astype(int)

# Avarege Time On Site by Date

In [13]:
df2.groupby(['date']).mean()['timeOnSite'].to_frame()

,timeOnSite
date,
20160801,219.714286
20160802,92.153846
20160803,384.818182
20160804,92.181818
20160805,133.187500
...,...
20170728,53.500000
20170729,123.800000
20170730,290.153846


# Number of Sessions by Date and Browser

In [14]:
df2.groupby(['date','browser']).count()['fullVisitorId'].to_frame()

fullVisitorId
date     browser                         
20160801 Chrome                        16
         Firefox                        2
         Safari                         3
20160802 Chrome                         9
         Safari                         4
...                                   ...
20170731 Opera Mini                     1
         Safari                         4
20170801 Chrome                        24
         Internet Explorer              1
         Safari                         6

[1489 rows x 1 columns]

# Number of distinct Sessions by Date

In [15]:
df2.groupby(['date']).count()['NewId'].to_frame()

,NewId
date,
20160801,21
20160802,13
20160803,22
20160804,33
20160805,16
...,...
20170728,20
20170729,15
20170730,13
